# Figure BSA_0885_FACS_FD

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 
                         
            # Data 
            library(tidyverse), 
            library(data.table), 
            library(reactable), 

            # Plotting 
            library(ComplexHeatmap), 
            library(patchwork), 
            library(cowplot), 
            library(ggrepel), 

            # Pyhton compatibility
            library(reticulate), 
            
            # Statistics
            library(emmeans)

        )
    )
)

In [ ]:
# Configure reticulate 
use_condaenv(condaenv="p.3.8.12-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO") # You need to adjust this path. I will help. 

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Load Seurat object

In [ ]:
so <- readRDS("data/BSA_0355_SM01_10x_SPLENO/ANALYSIS/pp.rds")

# RSS score 

In [ ]:
auc_mtx <- read.csv("result/scenic/auc_mtx.csv", row.names=1)
colnames(auc_mtx) <- gsub("\\.", "", colnames(auc_mtx))

In [ ]:
meta <- so@meta.data
meta <- meta[meta$cell_type_main %in% c("cMo", "PreRPM", "RPM"), ]

In [ ]:
meta$rss_group <- paste0(meta$cell_type_main, "|", meta$treatment)

In [ ]:
meta <- meta[intersect(rownames(meta), rownames(auc_mtx)), ]
auc_mtx <- auc_mtx[intersect(rownames(meta), rownames(auc_mtx)), ]

In [ ]:
# Load python libraries
pysenic_rss <- import("pyscenic.rss", convert=FALSE)
pd <- import("pandas", convert=FALSE)

In [ ]:
rss <- pysenic_rss$regulon_specificity_scores(auc_mtx, pd$Series(r_to_py(meta$rss_group))) %>% py_to_r()

In [ ]:
rss <- rss %>% tibble::rownames_to_column("rss_group")

In [ ]:
mat <- reshape2::melt(rss, id.vars=c("rss_group"), variable.name="module", value.name="rss") %>% 
    dplyr::mutate(cell_type_main=sapply(strsplit(rss_group, "\\|"), `[[`, 1), treatment=sapply(strsplit(rss_group, "\\|"), `[[`, 2)) %>%
    dplyr::mutate(treatment=ifelse(treatment=="NaCl", "rss_nacl", "rss_cpg")) %>%
    dcast(., cell_type_main + module ~ treatment, value.var="rss") %>% 
    dplyr::mutate(rss_diff=rss_cpg-rss_nacl) %>% 
    dplyr::mutate(treatment=ifelse(rss_diff>0, "CpG", "NaCl")) %>%

    # Rank RSS
    dplyr::group_by(cell_type_main) %>% 

    # Rank NaCl
    dplyr::arrange(cell_type_main, desc(rss_nacl)) %>% dplyr::mutate(rank_nacl=1:n()) %>% 
    dplyr::arrange(cell_type_main, desc(rss_cpg)) %>% dplyr::mutate(rank_cpg=1:n()) %>% 

    # Rank RSS diff
    dplyr::arrange(cell_type_main, desc(rss_diff)) %>% 
    dplyr::mutate(rank=1:n()) %>% dplyr::ungroup() %>% 
    dplyr::mutate(treatment=factor(treatment, levels=names(color$treatment)))

In [ ]:
dp_rss_top_label <- function(mat, rank_thr=250, rank_thr_label=100) {
    
    mat <- mat %>% dplyr::mutate(label=ifelse(rank %in% (mat %>% top_n(., wt=rss_nacl, rank_thr_label) %>% top_n(., wt=rss_diff, -15) %>% dplyr::pull(rank)), as.character(module), NA)) %>% 
    dplyr::mutate(label=ifelse(rank %in% (mat %>% top_n(., wt=rss_cpg, rank_thr_label) %>% top_n(., wt=rss_diff, 15) %>% dplyr::pull(rank)), as.character(module), label))
    
    mat <- dplyr::filter(mat, mat$rank_nacl <= rank_thr | mat$rank_cpg <= rank_thr)
    
    axis_min <- plyr::round_any(min(c(mat$rss_nacl, mat$rss_cpg)), accuracy=0.01, f=floor)
    axis_max <- plyr::round_any(max(c(mat$rss_nacl, mat$rss_cpg)), accuracy=0.01, f=ceiling)
    
    axis_breaks <- seq(axis_min, axis_max, by=0.04)
    
    ggplot(mat, aes(x=rss_nacl, y=rss_cpg, label=label, color=treatment)) + 
        geom_point(size=2) + 
        
        xlab("RSS CpG") + ylab("RSS NaCl") + ggtitle(mat$cell_type_main[1]) + 
        scale_x_continuous(limits=c(axis_min, axis_max), breaks=axis_breaks) + 
        scale_y_continuous(limits=c(axis_min, axis_max), breaks=axis_breaks) + 
        geom_abline(intercept=0, slope=1) + 
        scale_color_manual(values=color$treatment, name="Treatment") + 
        geom_text_repel(segment.color="black", force=10, force_pull=1, max.overlaps=getOption("ggrepel.max.overlaps", default=100), size=5, alpha=1, guide="none", segment.size=0.05, color="black", min.segment.length=0) + 
        guides(color=guide_legend(nrow=1, override.aes=list(alpha=1, size=3), keywidth=0, keyheight=0.25, default.unit="cm")) + 
        theme(aspect.ratio=1)

}

In [ ]:
options(repr.plot.width=3*5, repr.plot.height=5)

ggpubr::ggarrange(plotlist=lapply(split(mat, f=mat$cell_type_main), dp_rss_top_label), ncol=3, nrow=1, common.legend=TRUE, legend="bottom")